# **Step\#1: Setup reddit API**

In [ ]:
pip install --upgrade pip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install transformers
!pip install praw

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#%%
CLIENT_ID = 'X_pyX0Uh-dSgBsGXfC6ngw'
SECRET_TOKEN = 'M-DirjsphFa_l9nrvfo0DgSucgTh1w'
PASSWORD = "LucasBenjamin123"
#%%
import pandas as pd
import copy
#import requests
# import pandas as pd
# from datetime import datetime
# import re
# from textblob import TextBlob
# import torch

In [ ]:
#%%
import praw
#%%
reddit = praw.Reddit(client_id=CLIENT_ID,
                     client_secret=SECRET_TOKEN, password=PASSWORD,
                     user_agent='AnkiFlashCardCreator', username='DueFuel7692')


# **Step\#2: Use the object extracter transformer**

## Import HF models

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-large-finetuned-conll03-english")

model = AutoModelForTokenClassification.from_pretrained("xlm-roberta-large-finetuned-conll03-english")


### Make connection

In [ ]:
reddit = praw.Reddit(client_id=CLIENT_ID,
                     client_secret=SECRET_TOKEN, password=PASSWORD,
                     user_agent='MyBot2.0', username='DueFuel7692')


### Choose subreddit and filter posts


### Subreddit choice
several subreddits can be imputed here, instead of the hot posts, also the newest or the top posts could be retrieved by a changing the appropriate lines
e.g. subreddit: "nba", "news", "soccer", "machinelearning", "finance", etc.

In [ ]:
subreddit = reddit.subreddit('finance')

In [ ]:
list_of_post_titles = []
hot_posts = subreddit.hot(limit=10)
for submission in hot_posts:
  list_of_post_titles.append(submission.title)

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



In [ ]:
list_of_post_titles

['Moronic Monday - January 10, 2023 - Your Weekly Questions Thread',
 'CPI comes in at 6.5%, estimated 6.5%.',
 'Binance is bleeding Assets, $12 Billion Gone In Less Than 60 Days',
 'Swiss central bank posts biggest loss in its 116-year history',
 'The dull, high-risk business of yield farming among the European telcos — This time it’s different, maybe, apparently',
 'Citadel posts record revenues for hedge-fund, securities operations - WSJ',
 'Sam Bankman-Fried pleads not guilty to 8 counts of wire fraud, securities fraud, and conspiracy',
 'Moronic Monday - January 03, 2023 - Your Weekly Questions Thread',
 "Tough year ahead for the global economy, IMF's Georgieva warns",
 'If Risk Is Not Share Price Volatility... Then What Is It?']

#### Sentiment analysis
- Te lange titels verwijderen
- Gaan voor polariserende comment secties

### Create question dataframe

In [ ]:
classifier = pipeline("ner", model=model, tokenizer=tokenizer)
# objects_for_question = classifier(list_of_post_titles[4])


In [ ]:
def get_confident_objects(objects_for_question):
  for object in objects_for_question.copy():
    if object['score'] < 0.99:
      objects_for_question.remove(object)


  return objects_for_question

In [ ]:
#this loops over the list in reverse order
def get_answers(objects_for_question):
  copy_of_objects = copy.deepcopy(objects_for_question)

  list_of_final_words = []
  for i in range(len(copy_of_objects)-1, -1, -1):
    if (copy_of_objects[i]['word'].find('▁')) != 0:
      part_to_append = copy_of_objects[i]['word']
      #print(part_to_append)
      copy_of_objects[i-1]['word'] = copy_of_objects[i-1]['word'] + part_to_append
      #print('true')
    else: 
      list_of_final_words.append(copy_of_objects[i]['word'].replace('▁', ''))
  return list_of_final_words  


In [ ]:
def get_df_from_post_titles(list_of_post_titles):
  df = pd.DataFrame(columns = ['Answer','Context', 'SubjectNumber'])
  i = 0
  for context in list_of_post_titles:
    #get the different objects from the classifier
    objects_for_question = classifier(context)

    confident_objects = get_confident_objects(objects_for_question)

    answers = get_answers(confident_objects)
    i +=1

    
    for answer in answers:
      list_to_append = []
      list_to_append.append(answer)
      list_to_append.append(context)
      list_to_append.append(i)
      df.loc[len(df)] = list_to_append

  return df
      



  # for object in objects_for_question:
  #   list_with_answers.append(object['word'].replace('▁', ''))

In [ ]:
import nltk
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
intermediate_df = get_df_from_post_titles(list_of_post_titles)

In [ ]:

# Create a boolean mask indicating if the answer is a stop word
mask = intermediate_df['Answer'].apply(lambda x: x.lower() in stop_words)

# Drop the rows where the answer is a stop word
intermediate_df = intermediate_df[~mask]

In [ ]:
intermediate_df['list_of_words'] = intermediate_df['Context'].apply(lambda x: x.split(f' '))

In [ ]:
def check_position_difference(df):
    df['position'] = df.apply(lambda x: x['list_of_words'].index(x['Answer']) if x['Answer'] in x['list_of_words'] else None, axis=1)
    #df['check'] = df.apply(lambda x: x['position'] == x['position'].shift(-1) + 1 if x['position'] is not None else None, axis=1)
    return df

In [ ]:
check_position_difference(intermediate_df)

,Answer,Context,SubjectNumber,list_of_words,position
0,Moronic,"Moronic Monday - January 10, 2023 - Your Weekl...",1,"[Moronic, Monday, -, January, 10,, 2023, -, Yo...",0.0
1,Binance,"Binance is bleeding Assets, $12 Billion Gone I...",3,"[Binance, is, bleeding, Assets,, $12, Billion,...",0.0
2,Swiss,Swiss central bank posts biggest loss in its 1...,4,"[Swiss, central, bank, posts, biggest, loss, i...",0.0
3,European,"The dull, high-risk business of yield farming ...",5,"[The, dull,, high-risk, business, of, yield, f...",9.0
4,WSJ,"Citadel posts record revenues for hedge-fund, ...",6,"[Citadel, posts, record, revenues, for, hedge-...",9.0
5,Citadel,"Citadel posts record revenues for hedge-fund, ...",6,"[Citadel, posts, record, revenues, for, hedge-...",0.0
6,Bankman-Fried,Sam Bankman-Fried pleads not guilty to 8 count...,7,"[Sam, Bankman-Fried, pleads, not, guilty, to, ...",1.0
7,Sam,Sam Bankman-Fried pleads not guilty to 8 count...,7,"[Sam, Bankman-Fried, pleads, not, guilty, to, ...",0.0
8,Moronic,"Moronic Monday - January 03, 2023 - Your Weekl...",8,"[Moronic, Monday, -, January, 03,, 2023, -, Yo...",0.0
9,Georgieva,"Tough year ahead for the global economy, IMF's...",9,"[Tough, year, ahead, for, the, global, economy...",8.0


In [ ]:
intermediate_df.dropna(inplace = True)

In [ ]:
df = copy.deepcopy(intermediate_df)

In [ ]:
df

,Answer,Context,SubjectNumber,list_of_words,position
0,Moronic,"Moronic Monday - January 10, 2023 - Your Weekl...",1,"[Moronic, Monday, -, January, 10,, 2023, -, Yo...",0.0
1,Binance,"Binance is bleeding Assets, $12 Billion Gone I...",3,"[Binance, is, bleeding, Assets,, $12, Billion,...",0.0
2,Swiss,Swiss central bank posts biggest loss in its 1...,4,"[Swiss, central, bank, posts, biggest, loss, i...",0.0
3,European,"The dull, high-risk business of yield farming ...",5,"[The, dull,, high-risk, business, of, yield, f...",9.0
4,WSJ,"Citadel posts record revenues for hedge-fund, ...",6,"[Citadel, posts, record, revenues, for, hedge-...",9.0
5,Citadel,"Citadel posts record revenues for hedge-fund, ...",6,"[Citadel, posts, record, revenues, for, hedge-...",0.0
6,Bankman-Fried,Sam Bankman-Fried pleads not guilty to 8 count...,7,"[Sam, Bankman-Fried, pleads, not, guilty, to, ...",1.0
7,Sam,Sam Bankman-Fried pleads not guilty to 8 count...,7,"[Sam, Bankman-Fried, pleads, not, guilty, to, ...",0.0
8,Moronic,"Moronic Monday - January 03, 2023 - Your Weekl...",8,"[Moronic, Monday, -, January, 03,, 2023, -, Yo...",0.0
9,Georgieva,"Tough year ahead for the global economy, IMF's...",9,"[Tough, year, ahead, for, the, global, economy...",8.0


In [ ]:
to_remove = []

In [ ]:
df = df.reset_index().drop("index", axis = 1)

In [ ]:
for i in range(len(df)-1, 0, -1):
  #print(i)
  if df.iloc[i,2] == df.iloc[i-1, 2]:
    if df.iloc[i,4] == df.iloc[i-1, 4] - 1:
      df.iloc[i-1,0] = df.iloc[i,0] + " " +  df.iloc[i-1,0]
      to_remove.append(i)



In [ ]:
df.drop(to_remove, inplace = True, axis = 0)

In [ ]:
df

,Answer,Context,SubjectNumber,list_of_words,position
0,Moronic,"Moronic Monday - January 10, 2023 - Your Weekl...",1,"[Moronic, Monday, -, January, 10,, 2023, -, Yo...",0.0
1,Binance,"Binance is bleeding Assets, $12 Billion Gone I...",3,"[Binance, is, bleeding, Assets,, $12, Billion,...",0.0
2,Swiss,Swiss central bank posts biggest loss in its 1...,4,"[Swiss, central, bank, posts, biggest, loss, i...",0.0
3,European,"The dull, high-risk business of yield farming ...",5,"[The, dull,, high-risk, business, of, yield, f...",9.0
4,WSJ,"Citadel posts record revenues for hedge-fund, ...",6,"[Citadel, posts, record, revenues, for, hedge-...",9.0
5,Citadel,"Citadel posts record revenues for hedge-fund, ...",6,"[Citadel, posts, record, revenues, for, hedge-...",0.0
6,Sam Bankman-Fried,Sam Bankman-Fried pleads not guilty to 8 count...,7,"[Sam, Bankman-Fried, pleads, not, guilty, to, ...",1.0
8,Moronic,"Moronic Monday - January 03, 2023 - Your Weekl...",8,"[Moronic, Monday, -, January, 03,, 2023, -, Yo...",0.0
9,Georgieva,"Tough year ahead for the global economy, IMF's...",9,"[Tough, year, ahead, for, the, global, economy...",8.0


# **Step\#3: Generate the input and output for the flashcards**

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer2 = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap")

model2 = AutoModelForSeq2SeqLM.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap")

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


In [ ]:
def get_question(answer, context, max_length=250):
  input_text = "answer: %s  context: %s </s>" % (answer, context)
  features = tokenizer2([input_text], return_tensors='pt')

  output = model2.generate(input_ids=features['input_ids'], 
               attention_mask=features['attention_mask'],
               max_length=max_length)
  
  return_question = tokenizer2.decode(output[0]).split("<pad> question: ")[1].split("</s>")[0]


  return return_question

context = "Buffalo woman saves man with severe frostbite after getting him out of the storm and pleading for help in a Facebook livestream"
answer = "Facebook"

get_question(answer, context)

'What social network was the man pleading for help on?'

In [ ]:
df['Question'] = df.apply(lambda row: get_question(row['Answer'], row['Context']), axis=1)

In [ ]:
df

,Answer,Context,SubjectNumber,list_of_words,position,Question
0,Moronic,"Moronic Monday - January 10, 2023 - Your Weekl...",1,"[Moronic, Monday, -, January, 10,, 2023, -, Yo...",0.0,"What is the monday of January 10th, 2023?"
1,Binance,"Binance is bleeding Assets, $12 Billion Gone I...",3,"[Binance, is, bleeding, Assets,, $12, Billion,...",0.0,What company is bleeding Assets?
2,Swiss,Swiss central bank posts biggest loss in its 1...,4,"[Swiss, central, bank, posts, biggest, loss, i...",0.0,What is the largest loss in its 116-year history?
3,European,"The dull, high-risk business of yield farming ...",5,"[The, dull,, high-risk, business, of, yield, f...",9.0,What type of telcos do they operate in?
4,WSJ,"Citadel posts record revenues for hedge-fund, ...",6,"[Citadel, posts, record, revenues, for, hedge-...",9.0,What news outlet reported Citadel's record rev...
5,Citadel,"Citadel posts record revenues for hedge-fund, ...",6,"[Citadel, posts, record, revenues, for, hedge-...",0.0,What company posts record revenues for hedge-f...
6,Sam Bankman-Fried,Sam Bankman-Fried pleads not guilty to 8 count...,7,"[Sam, Bankman-Fried, pleads, not, guilty, to, ...",1.0,"Who pleads not guilty to wire fraud, securitie..."
8,Moronic,"Moronic Monday - January 03, 2023 - Your Weekl...",8,"[Moronic, Monday, -, January, 03,, 2023, -, Yo...",0.0,What is the name of Mondays?
9,Georgieva,"Tough year ahead for the global economy, IMF's...",9,"[Tough, year, ahead, for, the, global, economy...",8.0,Who warns of a tough year for the global economy?


In [ ]:
df['Quizlet Answer'] = df.apply(lambda row: row['Answer'] + f"    -    HEADLINE:\"" + row['Context'] + f"\"", axis=1)

In [ ]:
to_quizlet = df[["Question","Quizlet Answer"]]

In [ ]:
to_quizlet.to_excel("quizlet_finance.xlsx", index= False)

In [ ]:
import tokenizers

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer2 = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap")

model2 = AutoModelForSeq2SeqLM.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap")

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


In [ ]:

def get_question(answer, context, max_length=64):
  input_text = "answer: %s  context: %s </s>" % (answer, context)
  features = tokenizer2([input_text], return_tensors='pt')

  output = model2.generate(input_ids=features['input_ids'], 
               attention_mask=features['attention_mask'],
               max_length=max_length)

  return tokenizer2.decode(output[0])

context = "Buffalo woman saves man with severe frostbite after getting him out of the storm and pleading for help in a Facebook livestream"
answer = "Facebook"

get_question(answer, context)

'<pad> question: What social network was the man pleading for help on?</s>'